# Python for testers - REST

# What is REST?

* "representational state transfer"
* a software architectural style
* intended to simplify maintenance and yield high performace
* set of principles on how to use web standards
* typical application:
  * expose application functionality as readable URLs
  * use HTTP GET and POST to send queries
  * use JSON to exchange data
* see also: http://www.infoq.com/articles/rest-introduction

## JSON in Python

* "JavaScript object notation"
* human-readable text
* to exchange data objects
* attribute–value pairs
* available types: numbers, strings, boolean, array (list), object (dictionary), null (`None`)

## Convert Python to JSON

In [1]:
import json
persons = [
    {
        'name': 'Günter',
        'salary': 42000,
        'data_of_birth': '1976-07-02'
    },
    {
        'name': 'Alice',
        'salary': 47123.45,
        'date_of_birth': None
    },
]
json.dumps(persons)

'[{"name": "G\\u00fcnter", "salary": 42000, "data_of_birth": "1976-07-02"}, {"name": "Alice", "date_of_birth": null, "salary": 47123.45}]'

## Convert JSON to Python 

In [2]:
json.loads("""
[
    {"data_of_birth": "1976-07-02", "name": "G\\u00fcnter", "salary": 42000},
    {"date_of_birth": null, "name": "Alice", "salary": 47123.45}
]
""")

[{'data_of_birth': '1976-07-02', 'name': 'Günter', 'salary': 42000},
 {'date_of_birth': None, 'name': 'Alice', 'salary': 47123.45}]

## Convert JSON to Python with Decimal

By default, fractional numbers are converted to `float`. To use `Decimal` instead, specify `parse_float=Decimal`:

In [3]:
from decimal import Decimal
json.loads("""
[
    {"data_of_birth": "1976-07-02", "name": "G\\u00fcnter", "salary": 42000},
    {"date_of_birth": null, "name": "Alice", "salary": 47123.45}
]
""", parse_float=Decimal)

[{'data_of_birth': '1976-07-02', 'name': 'Günter', 'salary': 42000},
 {'date_of_birth': None, 'name': 'Alice', 'salary': Decimal('47123.45')}]

Note that `int` values like 42000 remain `int`.

# HTTP in Python

* The standard library provides modules for low level HTTP, in particular:
  * [urllib](https://docs.python.org/3/library/urllib.html).
  * [http](https://docs.python.org/3/library/http.htm)
* For high level client communication, use the external [requests](http://requests.readthedocs.org/) package.

# Example: Github issues

## Github issues

* Issue tracker for open source projects
* User view: https://github.com/roskakori/cutplace/issues
* API view: https://api.github.com/repos/roskakori/cutplace/issues?state=closed
* Returns a JSON result:
```
[
  {
    "url": "https://api.github.com/repos/roskakori/cutplace/issues/112",
    "repository_url": "https://api.github.com/repos/roskakori/cutplace",
    "labels_url": "https://api.github.com/repos/roskakori/cutplace/issues/112/labels{/name}",
    "id": 131714536,
    "number": 112,
    "title": "Opening a data file in UTF-16 with rows containing the € symbol ...",
...
```

## Send query

Send https://api.github.com/repos/roskakori/cutplace/issues?state=closed:

In [4]:
import requests
payload = {'state': 'closed'}
response = requests.get(
    'https://api.github.com/repos/roskakori/cutplace/issues',
    data=payload
)
response.status_code

200

## HTTP status code

* 1xx - informational
* 2xx - success
* 3xx - redirection
* 4xx - client error
* 5xx - server error
* For detailed code, see https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

## Access query data

In [5]:
response.text[:100]

'[{"url":"https://api.github.com/repos/roskakori/cutplace/issues/111","repository_url":"https://api.g'

In [6]:
issues = response.json(parse_float=Decimal)

most_recent_issue = issues[0]
most_recent_issue

{'assignee': {'avatar_url': 'https://avatars.githubusercontent.com/u/328726?v=3',
  'events_url': 'https://api.github.com/users/roskakori/events{/privacy}',
  'followers_url': 'https://api.github.com/users/roskakori/followers',
  'following_url': 'https://api.github.com/users/roskakori/following{/other_user}',
  'gists_url': 'https://api.github.com/users/roskakori/gists{/gist_id}',
  'gravatar_id': '',
  'html_url': 'https://github.com/roskakori',
  'id': 328726,
  'login': 'roskakori',
  'organizations_url': 'https://api.github.com/users/roskakori/orgs',
  'received_events_url': 'https://api.github.com/users/roskakori/received_events',
  'repos_url': 'https://api.github.com/users/roskakori/repos',
  'site_admin': False,
  'starred_url': 'https://api.github.com/users/roskakori/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/roskakori/subscriptions',
  'type': 'User',
  'url': 'https://api.github.com/users/roskakori'},
 'body': 'The joy. The thrill. The act

## Access JSON details

In [7]:
most_recent_issue['title']

'Migrate travis build from legacy to container-based infrastructure'

In [8]:
most_recent_issue['assignee']['login']

'roskakori'

## More requests features

* Cookies
* Proxies
* Customn athentication
* Streaming uploads
* ...

# Summary

* REST is easy in Python.
* Use `query = requests.get(..., data={...}` to send queries.
* Check for success on `query.status_code`.
* Access content using `query.json()`.
* Avoid floating point issues by specifying `parse_float=Decimal`.